In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('Prepared_data.csv')
df = df.set_index('Date')

In [3]:
dx = df.drop("signal",axis = 1)
dy = df["signal"]
X = dx
y = dy
print(X.shape)
print(y.shape)

(4110, 31)
(4110,)


In [4]:
dataset_length = df.shape[0]

split = int(dataset_length * 0.75)
split

3082

In [5]:
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(3082, 31) (1028, 31)
(3082,) (1028,)


In [6]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint

param_dist = {"max_depth": [3, None],
              "max_features": randint(1, 9),
              "min_samples_leaf": randint(1, 9),
              "criterion": ["gini", "entropy"]}
  
tree = RandomForestClassifier()
  
tree_cv = RandomizedSearchCV(tree, param_dist, cv = 5)
  
tree_cv.fit(X_train, y_train)

print("Best Parameters: {}".format(tree_cv.best_params_))
print("Best score is {}".format(tree_cv.best_score_))

Best Parameters: {'criterion': 'gini', 'max_depth': None, 'max_features': 6, 'min_samples_leaf': 4}
Best score is 0.934462943863268


In [7]:
criterion = tree_cv.best_params_['criterion']
max_depth = tree_cv.best_params_['max_depth']
max_features = tree_cv.best_params_['max_features']
min_samples_leaf = tree_cv.best_params_['min_samples_leaf']

In [8]:
clf = RandomForestClassifier(criterion = criterion,max_depth = max_depth,max_features = max_features,min_samples_leaf = min_samples_leaf,random_state=100)

In [9]:
model = clf.fit(X_train, y_train)

In [10]:
from sklearn.metrics import accuracy_score
print('Prediction (%): ', accuracy_score(y_test, model.predict(X_test), normalize=True)*100.0)

Prediction (%):  90.27237354085604


In [11]:
from sklearn.metrics import classification_report
report = classification_report(y_test, model.predict(X_test))
print(report)

              precision    recall  f1-score   support

           0       0.87      1.00      0.93       670
           1       1.00      0.72      0.84       358

    accuracy                           0.90      1028
   macro avg       0.94      0.86      0.88      1028
weighted avg       0.92      0.90      0.90      1028

